In [1]:
import tensorflow as tf
import numpy as np

In [8]:
num_elements = 10000
x1 = np.random.uniform(0,100, num_elements)
x2 = np.random.uniform(0,100, num_elements)
y = x1*x2
dataset = tf.data.Dataset.from_tensor_slices(({"x1":x1,"x2":x2}, y))
dataset = dataset.batch(100)

#for d, x in dataset.take(3):
#    print(d["x1"])


In [9]:
input1 = tf.keras.Input(shape=(1,), name="x1")
input2 = tf.keras.Input(shape=(1,), name="x2")

x1 = tf.keras.layers.Dense(6, activation='relu')(input1)
x2 = tf.keras.layers.Dense(6, activation='relu')(input2)
x = tf.keras.layers.Multiply()([x1,x2])
#x = tf.keras.layers.concatenate([x1, x2])
x = tf.keras.layers.Dense(6, activation='relu')(x)
x = tf.keras.layers.Dense(6, activation='relu')(x)
x = tf.keras.layers.Dense(6, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='relu', name='y')(x)

model = tf.keras.Model(inputs=[input1, input2], outputs=[output])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 x1 (InputLayer)                [(None, 1)]          0           []                               
                                                                                                  
 x2 (InputLayer)                [(None, 1)]          0           []                               
                                                                                                  
 dense_5 (Dense)                (None, 6)            12          ['x1[0][0]']                     
                                                                                                  
 dense_6 (Dense)                (None, 6)            12          ['x2[0][0]']                     
                                                                                            

In [10]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01),
    loss= tf.keras.losses.MeanSquaredError()
)

model.fit(dataset, epochs=100)

Epoch 1/100
100/100 [==============================] - 0s 686us/step - loss: 937011.6250
Epoch 2/100
100/100 [==============================] - 0s 665us/step - loss: 160.5886
Epoch 3/100
100/100 [==============================] - 0s 680us/step - loss: 114.8593
Epoch 4/100
100/100 [==============================] - 0s 700us/step - loss: 88.8115
Epoch 5/100
100/100 [==============================] - 0s 678us/step - loss: 65.6465
Epoch 6/100
100/100 [==============================] - 0s 679us/step - loss: 46.5034
Epoch 7/100
100/100 [==============================] - 0s 678us/step - loss: 31.6493
Epoch 8/100
100/100 [==============================] - 0s 684us/step - loss: 20.7962
Epoch 9/100
100/100 [==============================] - 0s 661us/step - loss: 13.2656
Epoch 10/100
100/100 [==============================] - 0s 675us/step - loss: 8.2487
Epoch 11/100
100/100 [==============================] - 0s 684us/step - loss: 5.0218
Epoch 12/100
100/100 [==============================] - 0s 

100/100 [==============================] - 0s 699us/step - loss: 0.0044
Epoch 98/100
100/100 [==============================] - 0s 678us/step - loss: 0.0042
Epoch 99/100
100/100 [==============================] - 0s 668us/step - loss: 0.0524
Epoch 100/100
100/100 [==============================] - 0s 679us/step - loss: 0.1475


In [11]:
# testing

exp = [tf.constant([2], tf.float32),tf.constant([3], tf.float32)]

print(model.predict(exp))

exp = [tf.constant([3], tf.float32),tf.constant([2], tf.float32)]

print(model.predict(exp))




[[6.131185]]
[[6.1310534]]


In [12]:
model.save("/home/david")

INFO:tensorflow:Assets written to: /home/david/assets


In [13]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("/home/david")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model,gpu_compatibility=True)


# Save the model.
with open("/home/david/x.tflite", 'wb') as f:
    f.write(tflite_model)
    
    
# Make sure the ids match with the ones used in training.
#labels = [{'name': 'base','id': 1},{'name': 'bar','id': 2},{'name': 'cover','id': 3}]

# Ensure that labels list is sorted.

#sorted_labels = sorted(labels, key=lambda d: d['id'])

#with open(files['OUTPUT_TFLITE_LABEL_MAP'], 'w') as f:
#    for label in sorted_labels:
#        name = label['name']
#        f.write(name + '\n')
        

#ImageSegmenterWriter = image_segmenter.MetadataWriter
#_MODEL_PATH = files['OUTPUT_TFLITE_MODEL']
# Task Library expects label files that are in the same format as the one below.

#_SAVE_TO_PATH = files['OUTPUT_TFLITE_MODEL_METADATA']
# Normalization parameters is required when preprocessing the image. It is
# optional if the image pixel values are in range of [0, 255] and the input
# tensor is quantized to uint8. See the introduction for normalization and
# quantization parameters below for more details.
# https://www.tensorflow.org/lite/convert/metadata#normalization_and_quantization_parameters)
#_INPUT_NORM_MEAN = 0
#_INPUT_NORM_STD = 1

#_LABEL_FILE = files['OUTPUT_TFLITE_LABEL_MAP']

#writer = ImageSegmenterWriter.create_for_inference(writer_utils.load_file(_MODEL_PATH), [_INPUT_NORM_MEAN], [_INPUT_NORM_STD],[_LABEL_FILE])


# Verify the metadata generated by metadata writer.
#print(writer.get_metadata_json())

# Populate the metadata into the model.
#writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)



=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the DEQUANTIZE op takes
tensor #7 as input and produces tensor #22 as output.

Subgraph#0 main(T#0, T#1) -> [T#20]
  Op#0 DEQUANTIZE(T#7) -> [T#22]
  Op#1 DEQUANTIZE(T#8) -> [T#29]
  Op#2 FULLY_CONNECTED(T#1, T#29, T#22) -> [T#14]
GPU COMPATIBILITY WARNING: FullyConnected doesn't support more than 2 runtime inputs.
  Op#3 DEQUANTIZE(T#9) -> [T#30]
  Op#4 DEQUANTIZE(T#6) -> [T#31]
  Op#5 FULLY_CONNECTED(T#0, T#30, T#31) -> [T#15]
GPU COMPATIBILITY WARNING: FullyConnected doesn't support more than 2 runtime inputs.
  Op#6 MUL(T#14, T#15) -> [T#16]
  Op#7 DEQUANTIZE(T#10) -> [T#23]
  Op#8 DEQUANTIZE(T#5) -> [T#32]
  Op#9 FULLY_CONNECTED(T#16, T#23, T#32) -> [T#17]
GPU COMPATIBILITY WARNING: FullyConnected doesn't support more than 2 runtime inputs.
  Op#10 DEQUANTIZE(T#4) -> [T#27]
  Op#11 DEQUANTIZE(T#11) -> [T#28]
  Op#12 F

2022-09-29 21:51:04.187519: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-09-29 21:51:04.187541: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-09-29 21:51:04.187686: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /home/david
2022-09-29 21:51:04.189178: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-09-29 21:51:04.189192: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /home/david
2022-09-29 21:51:04.192830: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-09-29 21:51:04.229513: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /home/david
2022-09-29 21:51:04.240491: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 52804 microseconds.
